In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install wandb

In [ ]:
!pip install sentencepiece

In [4]:
dir = 'drive/MyDrive/bhmw1blms/'

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/bhmw1blms')
from dataset import TextDataset

In [ ]:
from dataset import TextDataset

train_set = TextDataset(data_file=dir+'all_data.txt', data_file_for_tokenizer=dir+'data_for_tokenizer.txt', vocab_size=4000, train=True, sp_model_prefix=dir+'bpe', max_length=256)
valid_set = TextDataset(data_file=dir+'all_data.txt', data_file_for_tokenizer=dir+'data_for_tokenizer.txt', vocab_size=4000, train=False, sp_model_prefix=dir+'bpe', max_length=256)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, num_workers=12, shuffle=True, batch_size=512)
val_loader = DataLoader(valid_set, num_workers=12, shuffle=False, batch_size=512)

In [ ]:
from train import train, CosineAnnealingWithWarmupLR
from model import TransformerForStoryGeneration
import torch
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model7 = TransformerForStoryGeneration(train_set, embed_dim=512, num_heads=8, feedforward_dim=2048, num_layers=8, device=device).to(device)
# model7.load_state_dict(torch.load(dir+"model7_10_model.pth"))

In [ ]:
num_epochs_for_scheduler = 100
num_epochs = 100
warmup_steps = num_epochs_for_scheduler * len(train_loader) // 10 + 1
lr = 3e-4

optimizer = torch.optim.Adam(model7.parameters(), lr=lr, weight_decay=0.0)
# optimizer.load_state_dict(torch.load(dir+"model7_10_optimizer.pth"))

scheduler = CosineAnnealingWithWarmupLR(
    optimizer,
    warmup_steps=warmup_steps,
    max_steps=int(num_epochs_for_scheduler * len(train_loader))
)
# scheduler.load_state_dict(torch.load(dir+"model7_10_scheduler.pth"))

train(model=model7, optimizer=optimizer, scheduler=scheduler, train_loader=train_loader, val_loader=val_loader, num_epochs=num_epochs, model_name=dir+"model7", save_every=1, num_examples=5)